In [31]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)

In [3]:
df = pd.read_csv('datas/transactions.csv')

In [11]:
len(df)

11193677

In [4]:
# If same order_id products also have some same product_content_id, it means that customer bought different size of same item.
# Most probably, customer was planning to try these products and check if it fits or not. So not fitted products are strong candidate for returning back.
df['count'] = 1
df['same_prod_dif_size_count'] = df.groupby(['order_parent_id','product_content_id'])['count'].transform('count')

In [6]:
#last version prepared product dataframe
df_prod = pd.read_pickle('df_prods_last_embed_v1.pkl')

In [8]:
#check all products are in product dataframe
len(df.groupby(['product_content_id','product_variant_id'])) ,len(df_prod)

1457567

In [10]:
#merged dataframe has : production,transaction informations
df_prod_trans = pd.merge(df,df_prod,on=['product_content_id','product_variant_id'],how='inner')
len(df_prod_trans)

11193677

In [12]:
#For avoid memory problems
del df
del df_prod

In [13]:
#prepared user dataframe
df_user = pd.read_csv('datas/user_demographics.csv')

C:\Users\doguc\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
#merged dataframe has : production,transaction,user informations
df_prod_trans_user = pd.merge(df_prod_trans,df_user,on=['user_id'],how='inner')
len(df_prod_trans_user)

11193677

In [15]:
#df_prod_trans_user.to_pickle('df_prod_trans_user_v1.pkl')

In [18]:
len(df_prod_trans_user)

11193677

In [16]:
#For avoid memory problems
del df_prod_trans
del df_user

In [17]:
#prepared review dataframe
df_rew = pd.read_pickle('df_rew_groupby_prod_content_v3.pkl')

In [22]:
df_rew = df_rew.drop(columns=['product_content_id']).reset_index()

In [25]:
#merged dataframe has : production,transaction,user,review informations
df_prod_trans_user_rew = pd.merge(df_prod_trans_user,df_rew,on=['product_content_id'],how='left')

In [26]:
len(df_prod_trans_user_rew)

11193677

In [27]:
#For avoid memory problems
del df_prod_trans_user

In [28]:
#df_prod_trans_user_rew.to_pickle('df_prod_trans_user_rew_v1.pkl')

In [33]:
df_prod_trans_user_rew.head(1)

,order_date,user_id,is_elite_user,supplier_id_x,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned,count,same_prod_dif_size_count,product_id,brand_id,gender_id,category_id,color_id,attribute_value,embed_prod_name,embed_brand_name,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,birth_date,membership_date,gender,rate,review_like_count,supplier_id_y,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20,21,22,23,24,25,26,27,28,29
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.9,51.94,8.88,NaN,0.0,Sepette %35 İndirim,35.0,0,True,0.0,1,1,82687030,148,0,50,2,1180,"[-1.2849286, -1.2541153, -1.782976, -0.1273798...","[-0.023970377, -0.021582747, -0.024002993, -0....",-0.02397,-0.021583,-0.024003,-0.04902,-0.003887,-0.038962,-0.02406,-0.036875,0.049678,-0.006779,-0.038771,0.027583,-0.040332,-0.009561,0.013677,-0.035396,-0.043668,-0.033208,0.037342,0.045288,-1.284929,-1.254115,-1.782976,-0.12738,1.924536,0.055261,1.629931,-2.285416,-1.703261,-0.308678,1.169102,0.600538,-0.844449,-0.319412,-1.381115,-1.230004,-0.166193,0.662455,1.057862,-0.365327,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan,3.25,0.0,4662.0,-0.033843,-1.833734,-0.818248,-1.779563,-2.179461,-1.678068,-2.464959,0.605734,1.051787,0.692738,0.821323,-1.415752,1.587257,-0.264496,-1.374641,-0.596889,-0.273802,-2.635564,0.677161,-0.516057,1.706807,-0.245808,-0.395436,2.461663,-1.903484,0.331082,0.646957,-0.395696,-0.52306,-0.745091


In [35]:
#boolean column is converted to integer format.
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_prod_trans_user_rew['is_saved_card_trx'] = encoder.fit_transform(df_prod_trans_user_rew.is_saved_card_trx)

In [39]:
#df_prod_trans_user_rew.to_pickle('df_prod_trans_user_rew_v2.pkl')

In [37]:
#For avoid memory problems
del df_rew

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)
#pd.set_option('display.max_rows', 500)

In [2]:
df_main_concat = pd.read_pickle('df_prod_trans_user_rew_v2.pkl')

In [9]:
df_main_concat.head(1)

,order_date,user_id,is_elite_user,supplier_id_x,order_line_item_id,order_parent_id,product_content_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned,count,same_prod_dif_size_count,product_id,brand_id,gender_id,category_id,color_id,attribute_value,embed_prod_name,embed_brand_name,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,birth_date,membership_date,gender,rate,review_like_count,supplier_id_y,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20,21,22,23,24,25,26,27,28,29
0,2021-05-01 00:00:01.761000+00:00,3941815,0,4662,901280147,595758968,95503167,159147197,79.9,51.94,8.88,NaN,0.0,Sepette %35 İndirim,35.0,0,1,0.0,1,1,82687030,148,0,50,2,1180,"[-1.2849286, -1.2541153, -1.782976, -0.1273798...","[-0.023970377, -0.021582747, -0.024002993, -0....",-0.02397,-0.021583,-0.024003,-0.04902,-0.003887,-0.038962,-0.02406,-0.036875,0.049678,-0.006779,-0.038771,0.027583,-0.040332,-0.009561,0.013677,-0.035396,-0.043668,-0.033208,0.037342,0.045288,-1.284929,-1.254115,-1.782976,-0.12738,1.924536,0.055261,1.629931,-2.285416,-1.703261,-0.308678,1.169102,0.600538,-0.844449,-0.319412,-1.381115,-1.230004,-0.166193,0.662455,1.057862,-0.365327,1979-05-24 00:00:00+00:00,2011-04-07 14:55:39.457000+00:00,Bayan,3.25,0.0,4662.0,-0.033843,-1.833734,-0.818248,-1.779563,-2.179461,-1.678068,-2.464959,0.605734,1.051787,0.692738,0.821323,-1.415752,1.587257,-0.264496,-1.374641,-0.596889,-0.273802,-2.635564,0.677161,-0.516057,1.706807,-0.245808,-0.395436,2.461663,-1.903484,0.331082,0.646957,-0.395696,-0.52306,-0.745091


In [ ]:
# In this part, supplier rates are added as columns into main dataframe.

In [3]:
df_main_concat.rename(columns={'supplier_id_x':'supplier_id'},inplace=True)

In [4]:
df_main_concat = df_main_concat.drop(columns=['supplier_id_y','gender','order_date','user_id' ,'order_line_item_id' ,'coupon_id' ,'promotion_name' ,'product_id' ,'embed_prod_name'  ,'embed_brand_name','birth_date','membership_date'])

In [5]:
df_sup_ret = pd.read_csv('datas/supplier_return.csv')
df_sup_def = pd.read_csv('datas/supplier_defective_return.csv')
df_sup_dis = pd.read_csv('datas/supplier_disputed_return.csv')

In [6]:
df_sup_ret.rename(columns={'returnRate':'returnRate_ret'},inplace=True)
df_sup_def.rename(columns={'returnRate':'returnRate_def'},inplace=True)
df_sup_dis.rename(columns={'returnRate':'returnRate_dis'},inplace=True)

In [7]:
#All suppliers dataframes are merged.
df_ret_def = pd.merge(df_sup_ret,df_sup_def,on='supplier_id',how='inner')
df_ret_def_dis = pd.merge(df_ret_def,df_sup_dis,on='supplier_id',how='inner')
df_ret_def_dis.head()

,supplier_id,returnRate_ret,returnRate_def,total_claim,unresolved_claim,unresolved_accepted_claim,unresolvedclaim_percentage
0,247019,0.0,0.0,1,0,0,0.0
1,185253,0.0,0.0,1,0,0,0.0
2,193806,0.0,0.0,2,0,0,0.0
3,113616,0.0,0.0,1,0,0,0.0
4,345124,0.0,0.0,1,0,0,0.0


In [8]:
df_main_concat = pd.merge(df_main_concat,df_ret_def_dis,on=['supplier_id'],how='inner')